In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

from pprint import pprint
from zen_garden.postprocess.results import Results

### Working with results of a ZenGarden run

This notebook shows some examples on how to handle the results of a `zen-garden` run. It uses the outputs of the tests as examples. Therefore, you need to run the tests via 
```
coverage run --source="zen_garden" -m pytest -v tests/testcases/run_test.py
```
to get the data.

### Reading in the results

The results can be read in using the `Results` class. This will automatically read in all relevant files, independent of the number of scenarios or if the optimization was run with perfect or myopic foresight. However, you should know what type of input you are reading in, as the output of some routines will depend on the type.

In [3]:
# This has no scenarios and perfect foresight
res_basic = Results("../outputs/test_1a/")
# Here we have three different scenarios
res_scenario = Results("../outputs/test_6a/")
# This was run with myopic foresight
res_mf = Results("../outputs/test_4g/")
# This was run with aggregated time steps and multi year
res_ts = Results("../outputs/test_4f/")

### Accessing the config

The standard dictionaries, i.e. system, analysis, scenarios, solver and dict_sequence_time_steps can be accessed via the results dict.

In [4]:
# Show the analysis settings
pprint(res_basic.results["analysis"])

{'dataset': 'C:\\Users\\janis\\Dropbox\\SIS\\Subscriptions\\Sansavini\\ZEN-garden\\tests\\testcases\\test_1a',
 'discount_rate': 0.06,
 'folder_name_system_specification': 'system_specification',
 'folder_output': 'C:\\Users\\janis\\Dropbox\\SIS\\Subscriptions\\Sansavini\\ZEN-garden\\outputs',
 'header_data_inputs': {'set_capacity_types': 'capacity_type',
                        'set_carriers': 'carrier',
                        'set_conditioning_carrier_parents': 'carrier',
                        'set_conditioning_carriers': 'carrier',
                        'set_conversion_technologies': 'technology',
                        'set_dependent_carriers': 'carrier',
                        'set_edges': 'edge',
                        'set_elements': 'element',
                        'set_existing_technologies': 'existingTechnology',
                        'set_input_carriers': 'carrier',
                        'set_location': 'location',
                        'set_nodes': 'node',
 

### Extracting individual data frames

You can access individual data frames from any component with the `get_df` method. However, the behavior differs slightly for the usecases.
If the optimization was run with perfect foresight and no scenarios, then the data frame is returned as `pandas.Series`. For myopic foresight, the final dataframe is stiched together from the relevant outputs if the component is set yearly. If multiple scenarios were run, a dictionary containing the data frame for all different scenarios is returned. However, you can specify the scenario in advance, then only a single data frame is returned.

Additionally, the method provides a shortcut to save the extracted dataframe directly to a CSV, or in case of different scenarios to a bundle of CSV files.

In [5]:
# This will return a single data frame
df = res_basic.get_df('export_carrier_flow')
df

carrier      node  timeOperation
heat         CH    0                0.0
             DE    0                0.0
natural_gas  CH    0                0.0
             DE    0                0.0
Name: export_carrier_flow, dtype: float64

In [6]:
# you can also save the dataframe directly to a csv:
df = res_basic.get_df('carbon_emissions_overshoot', to_csv="test")
# this prints out the content of the generated csv
!cat test.csv

year,carbon_emissions_overshoot
0,0.0


In [7]:
# if we have multiple scenarios, all dataframe will be returned in a dict
df_dict = res_scenario.get_df('export_carrier_flow')
pprint(df_dict)

{'scenario_': carrier      node  timeOperation
heat         CH    0                0.0
                   1                0.0
                   2                0.0
                   3                0.0
             DE    0                0.0
                   1                0.0
                   2                0.0
                   3                0.0
natural_gas  CH    0                0.0
                   1                0.0
                   2                0.0
                   3                0.0
             DE    0                0.0
                   1                0.0
                   2                0.0
                   3                0.0
Name: export_carrier_flow, dtype: float64,
 'scenario_1': carrier      node  timeOperation
heat         CH    0                0.0
                   1                0.0
                   2                0.0
                   3                0.0
             DE    0                0.0
                   1  

In [8]:
# saving this to a csv will create a file for each scenario
df_dict = res_scenario.get_df('export_carrier_flow', to_csv="test2")
# list files and print content of one
!echo "Generated files: "
!ls test2*
!echo 
!echo "Output of test2_scenario_2.csv"
!cat test2_scenario_2.csv

Generated files: 
test2_scenario_.csv   test2_scenario_3.csv  test2_scenario_6.csv
test2_scenario_1.csv  test2_scenario_4.csv
test2_scenario_2.csv  test2_scenario_5.csv

Output of test2_scenario_2.csv
carrier,node,timeOperation,export_carrier_flow
heat,CH,0,0.0
heat,CH,1,0.0
heat,CH,2,0.0
heat,CH,3,0.0
heat,DE,0,0.0
heat,DE,1,0.0
heat,DE,2,0.0
heat,DE,3,0.0
natural_gas,CH,0,0.0
natural_gas,CH,1,0.0
natural_gas,CH,2,0.0
natural_gas,CH,3,0.0
natural_gas,DE,0,0.0
natural_gas,DE,1,0.0
natural_gas,DE,2,0.0
natural_gas,DE,3,0.0


In [9]:
# we can also specify a specific scneario
df = res_scenario.get_df('export_carrier_flow', scenario="scenario_2")
df

carrier      node  timeOperation
heat         CH    0                0.0
                   1                0.0
                   2                0.0
                   3                0.0
             DE    0                0.0
                   1                0.0
                   2                0.0
                   3                0.0
natural_gas  CH    0                0.0
                   1                0.0
                   2                0.0
                   3                0.0
             DE    0                0.0
                   1                0.0
                   2                0.0
                   3                0.0
Name: export_carrier_flow, dtype: float64

In [10]:
# This will collect the values from the results folders of the different myopic foresight runs
df = res_mf.get_df("carbon_emissions_cumulative")
df

0    151.293355
1    302.586710
2    453.880065
Name: carbon_emissions_cumulative, dtype: float64

### Helper functions

The `Results` class implements some helper functions. The function `get_total` allows you to calculate the total Value of a component, for a given year, element and scenario if desired. And the function `get_full_ts` calculates the full (disaggregate) time series for a given component.

In [11]:
# We read out a normal dataframe
df = res_mf.get_df("input_flow")
print(df)

technology          carrier      node  timeOperation
natural_gas_boiler  natural_gas  CH    0                 14.098413
                                       1                 14.098413
                                       2                 14.098413
                                       3                 28.516162
                                       4                 28.516162
                                       5                 28.516162
                                       6                  1.413877
                                       7                  1.413877
                                       8                  1.413877
                                       9                 21.433545
                                       10                21.433545
                                       11                21.433545
                                       12                 7.307499
                                       13                 7.307499
         

In [12]:
# We can calculate the total value of the dataframe
res_mf.get_total(df)

0              1  \
technology         carrier     node                                 
natural_gas_boiler natural_gas CH    110402.790806  110402.790806   
                               DE    646063.984500  646063.984500   

                                                 2  
technology         carrier     node                 
natural_gas_boiler natural_gas CH    110402.790806  
                               DE    646063.984500

In [13]:
# and also get the full timeseries (disaggregate) if we want
df_disaggregate = res_mf.get_full_ts(df)
df_disaggregate

0           1           2      \
natural_gas_boiler natural_gas CH   21.433545   21.433545   21.433545   
                               DE  117.280416  117.280416  117.280416   

                                        3           4           5      \
natural_gas_boiler natural_gas CH   25.004733   32.350333   32.350333   
                               DE  137.018935  186.971339  186.971339   

                                        6           7           8      \
natural_gas_boiler natural_gas CH   32.350333   32.350333   32.350333   
                               DE  186.971339  186.971339  186.971339   

                                        9      ...       26270       26271  \
natural_gas_boiler natural_gas CH   32.350333  ...   32.350333   32.350333   
                               DE  186.971339  ...  186.971339  186.971339   

                                        26272       26273       26274  \
natural_gas_boiler natural_gas CH   32.350333   32.350333   32.350333   
                               DE  186.971339  186.971339  186.971339   

                                        26275       26276       26277  \
natural_gas_boiler natural_gas CH   32.350333   32.350333   28.516162   
                               DE  186.971339  186.971339  160.082758   

                                        26278       26279  
natural_gas_boiler natural_gas CH   21.433545   21.433545  
                               DE  117.280416  117.280416  

[2 rows x 26280 columns]

### Some Plots

Here could be some example plots